# 0. Introduction

### 0.1 Description

The MNIST dataset is coming again. This time, the data contains information what `fashion` looks like. We are going to classify things like: T-Shirts, boots etc (The full list is below and inside the code...)

### 0.2 What does that data looks for humans...

In [1]:
%%html
<ul style = "list-style-type: none;">
    <li style = "float:left;"><img src ="./assets/1.png" width = 220px /></li>
    <li style = "float:left"><img src ="./assets/2.png" width = 220px /></li>
    <li style = "float:left"><img src ="./assets/3.png" width = 220px /></li>
    <li style = "float:left"><img src ="./assets/4.png" width = 220px /></li>
</ul>

### 0.3 Possible object labels

0 T-shirt/top
1 Trouser
2 Pullover
3 Dress
4 Coat
5 Sandal
6 Shirt
7 Sneaker
8 Bag
9 Ankle boot 

# 1. Imports of modules

In [2]:
"""Well-known library for uploading data & preprocessing"""
import pandas as pd

"""Visualising the data"""
import matplotlib.pyplot as plt


"""ML backend"""
import tensorflow as tf
import tensorflowjs as tfjs

from sklearn.model_selection import train_test_split

"""Keras dependencies for building model"""
import keras
from keras.models import Sequential
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.utils.np_utils import to_categorical 

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# 2. Preprocessing

### 2.1 Reading a dataset

In [3]:
dataframe = pd.read_csv("./data/fashion-mnist_train.csv") # 28x28 images, 784 pixels
dataframe_test = pd.read_csv("./data/fashion-mnist_test.csv")  # Same like above...

In [4]:
X = dataframe.iloc[:, 1:].values # Data to train & validate during training
Y = dataframe['label'].values #Labels

### 2.2 Scaling data/ Normalisation [0 - 1] range & Reshaping

In [5]:
X = X / 255
X = X.reshape(-1,28,28,1)
Y = to_categorical(Y, 10)

In [6]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size = 0.1)

### 2.3 Augmentation

In [7]:
data_generator = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=10,
    zoom_range = 0.1,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip= False,
    vertical_flip = False
)

data_generator.fit(X_train)

# 3. Building a model

### 3.1 Layers...

In [8]:
model = Sequential([Conv2D(32, (5,5),activation = "relu",padding = 'Same',input_shape = (28,28,1)),
            Conv2D(32, (5,5),activation = "relu",padding = 'Same'),
            MaxPool2D(pool_size = (2, 2)),
            Dropout(0.25),
            Conv2D(64, (3,3), activation = "relu", padding ='Same'),
            Conv2D(64, (3,3),activation = "relu", padding = 'Same'),
            MaxPool2D(pool_size = (2, 2), strides = (2,2)),
            Dropout(0.25),
            Flatten(),
            Dense(units = 256, activation = "relu"),
            Dense(units = 10, activation = "softmax") ])

### 3.2 Compilation, choosing loss function, optimizers and other parameters...

In [9]:
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [10]:
model.fit_generator(
    data_generator.flow(X_train, Y_train, batch_size = 128),
    epochs = 30,
    steps_per_epoch = X_train.shape[0] // 128,
    validation_data = (X_valid, Y_valid)
    )

Epoch 1/30
421/421 [==============================] - 382s 907ms/step - loss: 0.8230 - acc: 0.6914 - val_loss: 0.4968 - val_acc: 0.8045
Epoch 2/30
421/421 [==============================] - 392s 930ms/step - loss: 0.5706 - acc: 0.7841 - val_loss: 0.4339 - val_acc: 0.8312
Epoch 3/30
421/421 [==============================] - 386s 918ms/step - loss: 0.4931 - acc: 0.8139 - val_loss: 0.3667 - val_acc: 0.8618
Epoch 4/30
421/421 [==============================] - 396s 940ms/step - loss: 0.4420 - acc: 0.8345 - val_loss: 0.3196 - val_acc: 0.8805
Epoch 5/30
421/421 [==============================] - 394s 935ms/step - loss: 0.4063 - acc: 0.8471 - val_loss: 0.3153 - val_acc: 0.8820
Epoch 6/30
421/421 [==============================] - 395s 939ms/step - loss: 0.3851 - acc: 0.8550 - val_loss: 0.2879 - val_acc: 0.8913
Epoch 7/30
421/421 [==============================] - 394s 935ms/step - loss: 0.3748 - acc: 0.8599 - val_loss: 0.2980 - val_acc: 0.8885
Epoch 8/30
421/421 [============================

### 3.3 Saving to an external file

#### 3.3.1 .h5 

In [11]:
model.save("./model/model.h5")

#### 3.3.2 .json

In [12]:
tfjs.converters.save_keras_model(model, './model_json')